# Marascuillo.ipynb

https://www.itl.nist.gov/div898/handbook/prc/section4/prc46.htm


https://www.itl.nist.gov/div898/handbook/prc/section4/prc464.htm

In [1]:
import pandas as pd
import scipy.stats
import itertools
import math

In [2]:
# Test data

group = ['1','2','3','4','5']
bad = [36, 46, 42, 63, 38]
good = [264, 254, 258, 237, 262]
df = pd.DataFrame({'group': group, 'obs_bad': bad, 'obs_good': good})
df

,group,obs_bad,obs_good
0,1,36,264
1,2,46,254
2,3,42,258
3,4,63,237
4,5,38,262


# MY Data

In [3]:
group = ['CONTROL', 'DUG42', 'PNG', 'V23B', 'X2B']
OrNV_detected =     [1, 10, 4, 10, 4]
OrNV_not_detected = [34, 19, 12, 9, 13]
# df = pd.DataFrame({'group': group, 'obs_bad': OrNV_detected, 'obs_good': OrNV_not_detected})
df = pd.DataFrame({'group': group, 'OrNV_pos': OrNV_detected, 'OrNV_neg': OrNV_not_detected})
df

,group,OrNV_pos,OrNV_neg
0,CONTROL,1,34
1,DUG42,10,19
2,PNG,4,12
3,V23B,10,9
4,X2B,4,13


In [4]:
prop_OrNV_pos = df.OrNV_pos.sum() /(df.OrNV_pos.sum() + df.OrNV_neg.sum())
prop_OrNV_neg = 1 - prop_OrNV_pos
print(f'{prop_OrNV_pos=}   {prop_OrNV_neg=}')

prop_OrNV_pos=0.25   prop_OrNV_neg=0.75


In [5]:
# Calculate expected values and chi-squares

# df['n'] = df.obs_bad + df.obs_good
# df['p_bad'] = df.obs_bad / df.n
# df['exp_bad'] = prop_bad * df.n
# df['exp_good'] = prop_good * df.n
# df['chisq_bad'] = (df.obs_bad - df.exp_bad)**2 / df.exp_bad
# df['chisq_good'] = (df.obs_good - df.exp_good)**2 / df.exp_good
# df

df['n'] = df.OrNV_pos + df.OrNV_neg
df['p_OrNV_pos'] = df.OrNV_pos / df.n
df['exp_OrNV_pos'] = prop_OrNV_pos * df.n
df['exp_OrNV_neg'] = prop_OrNV_neg * df.n
df['chisq_OrNV_pos'] = (df.OrNV_pos - df.exp_OrNV_pos)**2 / df.exp_OrNV_pos
df['chisq_OrNV_neg'] = (df.OrNV_neg - df.exp_OrNV_neg)**2 / df.exp_OrNV_neg
df

,group,OrNV_pos,OrNV_neg,n,p_OrNV_pos,exp_OrNV_pos,exp_OrNV_neg,chisq_OrNV_pos,chisq_OrNV_neg
0,CONTROL,1,34,35,0.028571,8.75,26.25,6.864286,2.288095
1,DUG42,10,19,29,0.344828,7.25,21.75,1.043103,0.347701
2,PNG,4,12,16,0.250000,4.00,12.00,0.000000,0.000000
3,V23B,10,9,19,0.526316,4.75,14.25,5.802632,1.934211
4,X2B,4,13,17,0.235294,4.25,12.75,0.014706,0.004902


In [6]:
test_statistic = df.chisq_OrNV_pos.sum() + df.chisq_OrNV_neg.sum()
degrees_of_freedom = len(df) - 1
critical_value = scipy.stats.chi2.ppf(1-0.05, degrees_of_freedom)
print(f'{degrees_of_freedom=}   {critical_value=}   {test_statistic=}')

degrees_of_freedom=4   critical_value=9.487729036781154   test_statistic=18.299635498482516


In [7]:
# chi = math.sqrt(scipy.stats.chi2.ppf(0.975, degrees_of_freedom))
# print(f'{chi=}')

# pairs = list(itertools.combinations(group, 2))
# mylist = []
# for pair in pairs:
#     g1 = pair[0]
#     g2 = pair[1]
#     diff = abs(df[df.group==g1].p_bad.values[0] - df[df.group==g2].p_bad.values[0])
#     p1 = df[df.group==g1].p_bad.values[0]
#     n1 = df[df.group==g1].n.values[0]
#     p2 = df[df.group==g2].p_bad.values[0]
#     n2 = df[df.group==g2].n.values[0]
#     diff = abs(p1-p2)
#     critical_range = chi * math.sqrt(((p1 * (1 - p1)) / n1) + ((p2 * (1 - p2)) / n2)) 
#     if diff > critical_range:
#         significant = 'yes'
#     else:
#         significant = 'no'                
#     mylist.append({'g1': g1, 'g2': g2, 'diff': diff, 'critical_range': critical_range, 'significant': significant})
# pd.DataFrame(mylist)

chi = math.sqrt(scipy.stats.chi2.ppf(0.975, degrees_of_freedom))
print(f'{chi=}')

pairs = list(itertools.combinations(group, 2))
mylist = []
for pair in pairs:
    g1 = pair[0]
    g2 = pair[1]
    diff = abs(df[df.group==g1].p_OrNV_pos.values[0] - df[df.group==g2].p_OrNV_pos.values[0])
    p1 = df[df.group==g1].p_OrNV_pos.values[0]
    n1 = df[df.group==g1].n.values[0]
    p2 = df[df.group==g2].p_OrNV_pos.values[0]
    n2 = df[df.group==g2].n.values[0]
    diff = abs(p1-p2)
    critical_range = chi * math.sqrt(((p1 * (1 - p1)) / n1) + ((p2 * (1 - p2)) / n2)) 
    if diff > critical_range:
        significant = 'yes'
    else:
        significant = 'no'                
    mylist.append({'g1': g1, 'g2': g2, 'diff': diff, 'critical_range': critical_range, 'significant': significant})
pd.DataFrame(mylist)

chi=3.338156194949211


,g1,g2,diff,critical_range,significant
0,CONTROL,DUG42,0.316256,0.309269,yes
1,CONTROL,PNG,0.221429,0.373393,no
2,CONTROL,V23B,0.497744,0.393767,yes
3,CONTROL,X2B,0.206723,0.356060,no
4,DUG42,PNG,0.094828,0.466257,no
5,DUG42,V23B,0.181488,0.482728,no
6,DUG42,X2B,0.109533,0.452496,no
7,PNG,V23B,0.276316,0.526119,no
8,PNG,X2B,0.014706,0.498526,no
9,V23B,X2B,0.291022,0.513963,no
